# Sentiment analysis on movie reviews

The scope of this project is to classify movie reviews into positive or negative class. First it is performed at document level then at sentence level.

Download spacy english large corpus and load it without the unecessary pipeline processes

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.3 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=0865bd1ca5f332b3ec0cfefd1986894279bf983f1d9813eb8b018b7a2c09fb7a
  Stored in directory: /tmp/pip-ephem-wheel-cache-sbve9g2w/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


Import *en_core_web_lg*, this language model will be used to tokenize tokens and to split the reviews into single sentences.

In [ ]:
import en_core_web_lg
nlp = en_core_web_lg.load(disable=["tagger", "parser", "ner", "entity_linker", "entity_ruler", "textcat", "textcat_multilabel", "morphologizer", "attribute_ruler", "transformer"])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

Import used libraries

In [ ]:
import copy

import numpy as np
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchtext.datasets import IMDB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Function used to preprocess one review. It makes text lowercase, remove html tags that could still be present due to the web scraping phase, removes punctuation, removes stop words and lemmatize the text

In [ ]:
def data_preprocessing(text):
  text = text.lower()
  text = re.sub('<.*?>', '', text)  # Remove HTML from text
  text = ''.join([c for c in text if c not in string.punctuation])  # Remove punctuation
  text = [word for word in text.split() if word not in stop_words]
  text = ' '.join(text)
  return ' '.join([w.lemma_ for w in nlp(text)])

Function that vectorize the tokens of one review. The vecotorization used is implemented in the loaded spacy pipeline, it cosists of a vector of dimension 300 for each token.

In [ ]:
embedding_dim = 300  # the one used by spaCy


def embed_one_review(review):
  unknown = np.ones(embedding_dim, dtype=np.float32)
  embedded_data = []
  text = review["text"]
  tokens2vec = []
  for word in text.split():
    token2vec = nlp.vocab[word].vector.astype(np.float32)
    if np.count_nonzero(token2vec) == 0:
      token2vec = unknown
    tokens2vec.append(token2vec.tolist())
  review["text"] = tokens2vec
  return review

Function that adds padding to the review if it is too short or truncate if it is too long.

In [ ]:
chosen_length = 256

In [ ]:
def padding_one_review(review, seq_len):
  zero = list(np.zeros(embedding_dim, dtype=np.float32))
  if len(review["text"]) <= seq_len:
    zeros = [zero for each in range(seq_len - len(review["text"]))]
    new = zeros + review["text"]
  else:
    new = review["text"][: seq_len]
  new = {
    "text": new,
    "sentiment": review["sentiment"]
  }
  return new

Load dataset and store it in two dictionaries, one for training and one for testing, composed like so:


```
{
  "text": the text of the review,
  "sentiment": 0 or 1
}
```
0 is negative and 1 is positive


The dataset used is [IMDB Movie Reviews](https://pytorch.org/text/0.8.1/_modules/torchtext/datasets/imdb.html) provided by pytorch

In [ ]:
train_iter = IMDB(split='train')
train_data = []
for label, review in train_iter:
  review = data_preprocessing(review)
  review = {
    "text": review,
    "sentiment": 1 if label == "pos" else 0
  }
  train_data.append(review)
print(len(train_data))
print(train_data[0])

test_iter = IMDB(split='test')
val_data = []
for label, review in test_iter:
  review = data_preprocessing(review)
  review = {
    "text": review,
    "sentiment": 1 if label == "pos" else 0
  }
  val_data.append(review)
print(len(val_data))
print(val_data[0])

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 78.6MB/s]


25000
{'text': 'rent curiousyellow video store controversy surround \ufeff1 release 1967 also hear \ufeff1 seize us custom ever try enter country therefore fan film consider controversial really see myselfthe plot center around young swedish drama student name lena want learn everything life particular want focus attention make sort documentary average swede think certain political issue vietnam war race issue unite state ask politician ordinary denizen stockholm opinion politic sex drama teacher classmate marry menwhat kill curiousyellow 40 year ago consider pornographic really sex nudity scene far even shoot like cheaply make porno countryman mind find shock reality sex nudity major staple swedish cinema even ingmar bergman arguably answer good old boy john ford sex scene filmsi commend filmmakers fact sex show film show artistic purpose rather shock people make money show pornographic theater america curiousyellow good film anyone want study meat potato pun intend swedish cinema rea

Create dataset by inheriting from Dataset class from the pytorch library. The embedding and padding operations are performed during the get item because having in RAM all the embedded reviews isn't possible in google collab due to excessive memory usage.

In [ ]:
class SentimentDataset(Dataset):

  def __init__(self, reviews):
    self.reviews = reviews

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, idx):  # TODO if the bottleneck is here and we cannot do this operation before due to memory issues, try to do this operation only for the first epoch
    review = copy.deepcopy(self.reviews[idx])
    review = embed_one_review(review)
    review = padding_one_review(review, chosen_length)
    return torch.tensor(review["text"]), torch.tensor(review["sentiment"])

In [ ]:
# create tensor dataset
train_dataset = SentimentDataset(copy.deepcopy(train_data))
valid_dataset = SentimentDataset(copy.deepcopy(val_data))

# dataloaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False)

Create the model that will process the documents. It is a GRU stacked with two fully connected layers with a sigmoid as output. 

In [ ]:
class sentimentGRU(nn.Module):
  """
  The RNN model that will be used to perform Sentiment analysis.
  """

  def __init__(self, embedding_dim, hidden_dim, n_layers, drop_prob=0.5, output_size=1):
    """
    Initialize the model by setting up the layers.
    """
    super().__init__()

    self.output_size = output_size
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers

    # GRU layer
    self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True, bidirectional=True)

    # Dropout layer
    self.dropout = nn.Dropout(0.3)
    
    # Non linearity
    self.leaky_ReLU = nn.LeakyReLU(0.1)

    # Linear and sigmoid layers
    self.fc_in = nn.Linear(hidden_dim, hidden_dim)
    self.fc_out = nn.Linear(hidden_dim, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x, hidden):
    """
    Perform a forward pass of our model on some input and hidden state.
    """
    batch_size = x.size(0)

    # embedding and gru_out
    gru_out, hidden = self.gru(x, hidden)

    # stack up gru outputs
    gru_out = gru_out.contiguous().view(-1, self.hidden_dim)

    # Dropout and fully connected layer
    out = self.dropout(gru_out)
    out = self.fc_in(out)
    out = self.leaky_ReLU(out)
    out = self.dropout(out)
    out = self.fc_out(out)

    # sigmoid function
    sig_out = self.sigmoid(out)

    # reshape to be batch size first
    sig_out = sig_out.view(batch_size, -1)
    sig_out = sig_out[:, -1] # get last batch of labels

    return sig_out, hidden

  def init_hidden(self, batch_size):
    """
    Initializes hidden state
    """
    # Create a tensor with sizes n_layers x batch_size x hidden_dim,
    # initialized to zero, for hidden state
    # number of layers is multiplied by 2 because it is bidirectional GRU
    h0 = torch.zeros((self.n_layers*2, batch_size,self.hidden_dim)).to(device)
    hidden = h0
    return hidden

Instantiate the model with 3 layers in the GRU and 256 as hidden dimensions, these values have been chosen heuristically

In [ ]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("GPU not available, CPU used")

GPU is available


In [ ]:
# Instantiate the model w/ hyperparams
output_size = 1
hidden_dim = 256
n_layers = 3

model = sentimentGRU(embedding_dim, hidden_dim, n_layers, output_size=output_size)
model = model.to(device)

print(model)

sentimentGRU(
  (gru): GRU(300, 256, num_layers=3, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (leaky_ReLU): LeakyReLU(negative_slope=0.1)
  (fc_in): Linear(in_features=256, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


Define training function

In [ ]:
def train(model, train_loader, batch_size, criterion, optimizer):
  train_losses = []
  train_acc = 0.0
  model.train()
  # initialize hidden state 
  h = model.init_hidden(batch_size).to(device)

  # store data for f1 score
  true_positive = 0
  false_negative = 0
  all_positive = 0
  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)   
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    if inputs.size(0) < batch_size:
      h = model.init_hidden(inputs.size(0)).to(device)
    # h = tuple([each.data for each in h])
    h = h.detach().clone().to(device)
    output, h = model(inputs, h)

    # calculate the loss and perform backprop
    loss = criterion(output.squeeze(), labels.float())
    loss.backward()
    train_losses.append(loss.item())
    # calculating accuracy
    accuracy = acc(output,labels)
    train_acc += accuracy
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    optimizer.zero_grad()
    for output_idx, pred in enumerate(output):
      pred = int(torch.round(pred).item())
      label = labels[output_idx].item()
      if label == 1 and pred == label:
        true_positive += 1
      if pred == 0 and label == 1:
        false_negative += 1
      if pred == 1:
        all_positive += 1

  precision = true_positive / all_positive if all_positive != 0 else 0
  recall = true_positive / (true_positive + false_negative)
  f1_score_train = 2 * (precision * recall) / (precision + recall)
  return train_losses, train_acc, f1_score_train

Define evaluation function

In [ ]:
def eval(model, valid_loader, batch_size, criterion):
  true_positive = 0
  false_negative = 0
  all_positive = 0
  val_h = model.init_hidden(batch_size).to(device)
  val_losses = []
  val_acc = 0.0
  model.eval()
  for inputs, labels in valid_loader:
    # val_h = tuple([each.data for each in val_h])
    val_h = val_h.detach().clone().to(device)
    inputs, labels = inputs.to(device), labels.to(device)
    if inputs.size(0) < batch_size:
      val_h = model.init_hidden(inputs.size(0)).to(device)

    output, val_h = model(inputs, val_h)
    val_loss = criterion(output.squeeze(), labels.float())

    val_losses.append(val_loss.item())
        
    accuracy = acc(output,labels)
    val_acc += accuracy
        
    for output_idx, pred in enumerate(output):
      pred = int(torch.round(pred).item())
      label = labels[output_idx].item()
      if label == 1 and pred == label:
        true_positive += 1
      if pred == 0 and label == 1:
        false_negative += 1
      if pred == 1:
        all_positive += 1
  precision = true_positive / all_positive if all_positive != 0 else 0
  recall = true_positive / (true_positive + false_negative)
  f1_score_val = 2 * (precision * recall) / (precision + recall)
  return val_losses, val_acc, f1_score_val

## Start the training and evaluation loop

The following code initializes the optimizer, loss function and all the variables needed to compute accuracy and f1 score.

The default number of epochs is 5 because I empirically seen that after the fourth epoch the network start to overfit.

A mecanism of early stopping is implemented by storing the weights of the network in a variable when the validation f1 score is greater than the current maximum. So only the weights that perform better on the validation dataset are kept.

In [ ]:
saved_model_state_dict = None
criterion = nn.BCELoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)


# function to compute accuracy
def acc(pred,label):
  pred = torch.round(pred.squeeze())
  return torch.sum(pred == label.squeeze()).item()

clip = 5
epochs = 5
valid_loss_min = np.Inf
f1_score_max = 0

# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]


for epoch in range(epochs):
  train_losses, train_acc, f1_score_train = train(model, train_loader, batch_size, criterion, optimizer)

  val_losses, val_acc, f1_score_val = eval(model, valid_loader, batch_size, criterion)

  epoch_train_loss = np.mean(train_losses)
  epoch_val_loss = np.mean(val_losses)
  epoch_train_acc = train_acc/len(train_loader.dataset)
  epoch_val_acc = val_acc/len(valid_loader.dataset)
  epoch_tr_loss.append(epoch_train_loss)
  epoch_vl_loss.append(epoch_val_loss)
  epoch_tr_acc.append(epoch_train_acc)
  epoch_vl_acc.append(epoch_val_acc)
  print(f'Epoch {epoch+1}') 
  print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
  print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')

  
  print(f"f1 score training: {f1_score_train} f1 score validation: {f1_score_val}")

  if f1_score_val >= f1_score_max:
    print('f1 score increased ({:.6f} --> {:.6f}).  Saving model ...'.format(f1_score_max,f1_score_val))
    saved_model_state_dict = model.state_dict()
    f1_score_max = f1_score_val
  print(40*'==')

Epoch 1
train_loss : 0.4925702843739062 val_loss : 0.333573760122669
train_accuracy : 76.34 val_accuracy : 85.66
f1 score training: 0.7666574618328139 f1 score validation: 0.8574949318281194
f1 score increased (0.000000 --> 0.857495).  Saving model ...
Epoch 2
train_loss : 0.32122921251824926 val_loss : 0.3176987685111104
train_accuracy : 86.44800000000001 val_accuracy : 86.70400000000001
f1 score training: 0.864804469273743 f1 score validation: 0.857375783060156
Epoch 3
train_loss : 0.28345000188873737 val_loss : 0.2903445279415773
train_accuracy : 88.444 val_accuracy : 87.652
f1 score training: 0.8847488730203056 f1 score validation: 0.8700593509281475
f1 score increased (0.857495 --> 0.870059).  Saving model ...


KeyboardInterrupt: ignored

# Sentence level document classification

Import the necessary libraries

In [ ]:
import numpy
from sklearn.metrics import classification_report

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Define a function that preprocess the reviews, it removes every HTML tags left from the web scraping phase

In [ ]:
def sentence_preprocessing(text):
  text = re.sub('<.*?>', '', text)  # Remove HTML from text
  return text

In [ ]:
vaderAnalyzer = SentimentIntensityAnalyzer()

sentiments = []
reviews = []
for label, review in IMDB(split='test'):
  if label == "neg":
    sentiments.append(0)
  else:
    sentiments.append(1)
  reviews.append(sentence_preprocessing(review))

In [ ]:
list_of_sentences = []
for review_idx, review in enumerate(reviews):
  list_of_sentences.append([])
  for sent in nlp(review).sents:
    list_of_sentences[review_idx].append(sent.text)
print(list_of_sentences[0])

['I love sci-fi and am willing to put up with a lot.', 'Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood.', 'I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original).', "Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (", "I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV.", "It's not.", "It's clichéd and uninspiring.)", 'While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf.', 'Star Trek).', 'It may treat important issues, yet not as a serious philosophy.', "It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life.", 'Their actions and reactions are wooden and predictable, often painful to watch.', 'The makers of Earth 

**Classification**

Two approaches has been tried, befre the classification phase the list of list of scores is stored.

This first method classify each sentence into positive or negative by looking at the score given by VADER. Then the entire review is classified as positive if the number of negative sentences is less then the number of positive ones.

In [ ]:
scores_predicted = []
for review_idx, doc in enumerate(list_of_sentences):
  scores_predicted.append([])
  for sent in doc:
    scores_predicted[review_idx].append(vaderAnalyzer.polarity_scores(sent))


sentiments_predicted_text = []
for review_idx, doc in enumerate(scores_predicted):
  sentiments_predicted_text.append([])
  for sent_score in doc:
    if sent_score['neg'] > sent_score['pos']:
      sentiments_predicted_text[review_idx].append('neg')
    else:
      sentiments_predicted_text[review_idx].append('pos')


sentiments_predicted = []
for doc in sentiments_predicted_text:
  if doc.count('neg') > doc.count('pos'):
    sentiments_predicted.append(0)
  else:
    sentiments_predicted.append(1)

print(classification_report(sentiments, sentiments_predicted))

              precision    recall  f1-score   support

           0       0.83      0.25      0.38     12500
           1       0.56      0.95      0.70     12500

    accuracy                           0.60     25000
   macro avg       0.70      0.60      0.54     25000
weighted avg       0.70      0.60      0.54     25000



This second approach takes into account of the actual value of the score given by VADER and sum all of them to get a final score for all positive contribution and negative contribution. If the summed positive scores is greater than the summed negative the review is considered as positive.

This approach allows me to not account for objective sentences, because objective sentences have very low positve and negative values, actually giving poor contribution to the classification, hence a great bust in performance with respect the previous attempt with V.A.D.E.R.

In [ ]:
sentiments_predicted = []
for doc in scores_predicted:
  pos = 0.0
  neg = 0.0
  for sent_score in doc:
    pos += sent_score['pos']
    neg += sent_score['neg']
  sentiments_predicted.append(0 if neg > pos else 1)

print(classification_report(sentiments, sentiments_predicted))

              precision    recall  f1-score   support

           0       0.82      0.55      0.66     12500
           1       0.66      0.88      0.75     12500

    accuracy                           0.71     25000
   macro avg       0.74      0.71      0.71     25000
weighted avg       0.74      0.71      0.71     25000

